In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
df = pd.read_csv(f"../data/VN30F1M-20230626.csv")

In [3]:
df

,SYMBOL,TRADING_DATE,OPEN_PRICE/1000,CLOSE_PRICE/1000,HIGH_PRICE/1000,LOW_PRICE/1000,OPEN_INTEREST,VOLUME,BUY_VOLUME,SELL_VOLUME
0,VN30F1M,20230626 13:03:40,1114.7,1114.2,1114.7,1114.2,50146,312,279,33
1,VN30F1M,20230626 13:02:57,1115.7,1114.9,1115.7,1114.9,50146,419,386,33
2,VN30F1M,20230626 13:01:59,1114.9,1115.8,1115.9,1114.8,50146,895,132,763
3,VN30F1M,20230626 13:00:57,1114.9,1114.8,1114.9,1114.4,50146,345,155,189
4,VN30F1M,20230626 11:30:06,1115.0,1114.9,1115.0,1114.9,50146,9,0,1
...,...,...,...,...,...,...,...,...,...,...
340940,VN30F1M,20171106 09:03:54,841.6,842.0,842.0,841.6,3336,56,36,20
340941,VN30F1M,20171106 09:02:57,841.1,841.5,841.5,841.1,3336,37,15,22
340942,VN30F1M,20171106 09:01:48,841.0,841.0,841.0,840.7,3336,43,35,8
340943,VN30F1M,20171106 09:00:51,841.0,840.6,841.7,840.6,3336,253,243,10


In [4]:
df.dtypes

SYMBOL               object
TRADING_DATE         object
OPEN_PRICE/1000     float64
CLOSE_PRICE/1000    float64
HIGH_PRICE/1000     float64
LOW_PRICE/1000      float64
OPEN_INTEREST         int64
VOLUME                int64
BUY_VOLUME            int64
SELL_VOLUME           int64
dtype: object

### Wrangling & Standardizing Dates

Create new columns to clean up trading date and convert to pandas timestamp

In [5]:
df.columns = df.columns.str.lower()

In [6]:
# The .str accessor allows us to apply string operations to each element of the column
# astype(int) method to convert the resulting string values to integers. 
df["year"] = df["trading_date"].str[:4].astype(int)
df["month"] = df["trading_date"].str[4:6].astype(int)
df["day"] = df["trading_date"].str[6:8].astype(int)
df["hour"] = df["trading_date"].str[9:11].astype(int)
df["minute"] = df["trading_date"].str[12:14].astype(int)
df["second"] = df["trading_date"].str[15:17].astype(int)

In [7]:
datetime = pd.to_datetime(df[['year', 'month', 'day', 'hour', 'minute', 'second']])

#Round the datetime to the nearest minute
rounded_datetime = pd.Series(datetime).dt.round("min")

print(rounded_datetime)

0        2023-06-26 13:04:00
1        2023-06-26 13:03:00
2        2023-06-26 13:02:00
3        2023-06-26 13:01:00
4        2023-06-26 11:30:00
                 ...        
340940   2017-11-06 09:04:00
340941   2017-11-06 09:03:00
340942   2017-11-06 09:02:00
340943   2017-11-06 09:01:00
340944   2017-11-06 09:00:00
Length: 340945, dtype: datetime64[ns]


In [8]:
# Set datetime as index
df.set_index(rounded_datetime, inplace = True)

# Adjust hour, min to reflect new rounded. Drop 'second' column
df["hour"] = df.index.hour
df["minute"] = df.index.minute
df = df.drop("second", axis="columns")

Create a template that has sequential minute index at desirable timeframe

In [9]:
# Define the start and end dates
start_date = pd.to_datetime('2017-11-06').replace(hour=9, minute=0)
end_date = pd.to_datetime('2023-06-23').replace(hour=14, minute=30)

# Generate the date range for each day
dates = pd.date_range(start=start_date, end=end_date, freq='D')

# Create an empty list to store the datetime series
datetime_series = []

# Iterate over each day in the date range
for date in dates:
    # Generate the datetime series for each day between 9AM and 2:30PM
    morning_range = pd.date_range(start=date, end=date.replace(hour=11, minute=30), freq='min')
    afternoon_range = pd.date_range(start=date.replace(hour=13,minute=0), end=date.replace(hour=14, minute=30), freq='min')
    datetime_series.extend(morning_range)
    datetime_series.extend(afternoon_range)

# Create a new template dataframe and set datetime series as index
template_df = pd.DataFrame({'datetime': datetime_series})
template_df.set_index('datetime', inplace=True)

# Print the new DataFrame
template_df

""
datetime
2017-11-06 09:00:00
2017-11-06 09:01:00
2017-11-06 09:02:00
2017-11-06 09:03:00
2017-11-06 09:04:00
...
2023-06-23 14:26:00
2023-06-23 14:27:00
2023-06-23 14:28:00


In [10]:
# Perform left join
df = template_df.merge(df, how='left', left_index=True, right_index=True)

In [11]:
df["datetime"] = df.index

In [12]:
# Group the DataFrame by the date component of the datetime index
grouped = df.groupby(df.index.date)

# Fill NaN rows with preceding non-NaN values within each group
df = grouped.apply(lambda x: x.fillna(method='ffill'))

df = df.reset_index(drop=True)
df.set_index("datetime", inplace=True)

Create a column to show the number of days until maturity of future contracts

In [13]:
#Group the DataFrame by date
groups = df.groupby(df.index.date)
days_to_maturity = pd.Series()

for date, group in groups:
    this_month_beg = group.index[0] - pd.offsets.MonthBegin()
    this_month_end = this_month_beg + pd.offsets.MonthEnd()
    next_month_beg = group.index[0] + pd.offsets.MonthBegin()
    next_month_end = next_month_beg + pd.offsets.MonthEnd()
    
    third_thurs = pd.date_range(start = this_month_beg, end = this_month_end, freq = 'WOM-3THU')[0]

    if group.index[0] <= third_thurs:
        maturity = third_thurs
    else:
        maturity = pd.date_range(start = next_month_beg, end = next_month_end, freq = 'WOM-3THU')[0]
    
    days_diff = pd.Series(-(group.index - maturity).days)
        
    days_to_maturity = pd.concat([days_to_maturity, days_diff])

Dropping all rows that still has NaN values, drop duplicates and rename & drop unnecessary columns

In [14]:
df = df.dropna(axis=0, how="any")

Renaming & drop unncessary columns

In [15]:
df.rename(columns = {"open_price/1000": "open", "close_price/1000": "close", "high_price/1000": "high", "low_price/1000": "low", "trading_date": "raw_date", "open_interest" : "eod_open_interest"}, inplace=True)

df = df.drop(axis=1, labels=["year", "month", "day", "hour", "minute"])

/var/folders/sh/hbp85fl56x95nw9ps4scnxlc0000gn/T/ipykernel_1671/4018561994.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.rename(columns = {"open_price/1000": "open", "close_price/1000": "close", "high_price/1000": "high", "low_price/1000": "low", "trading_date": "raw_date", "open_interest" : "eod_open_interest"}, inplace=True)


In [16]:
# Check for duplicates in the index
duplicates_index = df.index.duplicated(keep='last')

df = df[~duplicates_index]

In [17]:
# Checking the number of data rows
num_days = pd.Series(df.index.date).nunique()
print(f"The data series has {num_days} days")
print(f"The data series has {df.shape[0]} periods")
print(f"Each day the series has {round(df.shape[0] / num_days)} rows")

The data series has 1407 days
The data series has 339079 periods
Each day the series has 241 rows


### Create new features to mark dates & trading value

In [18]:
df['day_of_week'] = df.index.day_of_week
df['day_of_year'] = df.index.day_of_year
df['is_month_end'] = df.index.is_month_end.astype(int)
df['is_month_start'] = df.index.is_month_start.astype(int)

In [19]:
df['is_morning'] = np.where((df.index.hour < 12), 1, 0)

In [25]:
df['est_average'] = (df["open"] + df["close"])/2
df['est_trading_val'] = df['volume'] * df['est_average']

In [21]:
# Get cumulative trading value and volume
df["cum_sum_tradingval"] = df["est_trading_val"].groupby(pd.Grouper(freq='D')).cumsum()
df["cum_sum_volume"] = df["volume"].groupby(pd.Grouper(freq='D')).cumsum()
df["cum_buy_volume"] = df["buy_volume"].groupby(pd.Grouper(freq='D')).cumsum()
df["cum_sell_volume"] = df["sell_volume"].groupby(pd.Grouper(freq='D')).cumsum()

In [24]:
df.columns

Index(['symbol', 'raw_date', 'open', 'close', 'high', 'low',
       'eod_open_interest', 'volume', 'buy_volume', 'sell_volume',
       'day_of_week', 'day_of_year', 'is_month_end', 'is_month_start',
       'is_morning', 'est_trading_val', 'cum_sum_tradingval', 'cum_sum_volume',
       'cum_buy_volume', 'cum_sell_volume'],
      dtype='object')

### Resampling to get other time intervals

In [26]:
aggregations = {
    'open': 'last',
    'close': 'last',
    'high' : 'max',
    'low' : 'min',
    'eod_open_interest' : 'last',
    'volume' : 'sum',
    'buy_volume': 'sum',
    'sell_volume': 'sum',
    'day_of_week': 'last',
    'day_of_year': 'last',
    'is_month_end': 'last',
    'is_month_start': 'last',
    'is_morning': 'last',
    'est_trading_val': 'sum',
    'est_average': 'mean',
    'cum_sum_tradingval': 'last',
    'cum_sum_volume': 'last',
    'cum_buy_volume': 'last',
    'cum_sell_volume': 'last',
}

df_1min = df
df_5min = df.resample('5min', label='right', closed='right').agg(aggregations)
df_10min = df.resample('10min', label='right', closed='right').agg(aggregations)
df_15min = df.resample('15min', label='right', closed='right').agg(aggregations)
df_30min = df.resample('30min', label='right', closed='right').agg(aggregations)

In [27]:
df_5min.dropna(axis=0, how='any', inplace=True)
df_10min.dropna(axis=0, how='any', inplace=True)
df_15min.dropna(axis=0, how='any', inplace=True)
df_30min.dropna(axis=0, how='any', inplace=True)

In [28]:
df_1min.to_csv(r"../data/VN30F1M_1min_cleaned.csv")
df_5min.to_csv(r"../data/VN30F1M_5min_cleaned.csv")
df_10min.to_csv(r"../data/VN30F1M_10min_cleaned.csv")
df_15min.to_csv(r"../data/VN30F1M_15min_cleaned.csv")
df_30min.to_csv(r"../data/VN30F1M_30min_cleaned.csv")